<a href="https://colab.research.google.com/github/valid999/NLP_Pytorch/blob/main/Bidirectiona_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bidirectional Many to one RNN

In [17]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


#  Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameter

sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_classes = 10
batch_size = 100
num_epochs = 5
learning_rate = 0.003

# Mnist dataset
train_dataset = torchvision.datasets.MNIST(root = '../../data/',
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True
                                           )

test_dataset = torchvision.datasets.MNIST(root = '../../data/',
                                          train = False,
                                          transform = transforms.ToTensor())


# Data loader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset ,
                                           batch_size = batch_size ,
                                           )

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size ,
                                          shuffle = False
                                          )


# # Bidirectional recurrent neural network (many-to-one)
class BiRNN(nn.Module):
  def __init__(self , input_size , hidden_size , num_layers , num_classes):
    super(BiRNN , self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size , hidden_size , num_layers , batch_first = True , bidirectional = True)
    self.fc = nn.Linear(hidden_size*2 , num_classes) # 2 for bidirectional

  def forward(self, x):
    # Set the initial states
    h0 = torch.zeros(self.num_layers*2 , x.size(0) , self.hidden_size).to(device) # 2 for bidirectional
    c0 = torch.zeros(self.num_layers*2 , x.size(0) , self.hidden_size).to(device)

    # Forward propagate LSTM
    out , _ = self.lstm(x , (h0 , c0)) # out: tensor of shape (batch_size , seq_length , hidden_size*2)

    # Decode the hidden state of thed last time step
    out = self.fc(out[: , -1 , :])
    return out


model = BiRNN(input_size , hidden_size , num_layers , num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters() , lr = learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i , (images , labels) in enumerate(train_loader):
    images = images.reshape(-1 , sequence_length , input_size).to(device)
    labels = labels.to(device)


    # forward pass
    outputs = model(images)
    loss = criterion(outputs , labels)


    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
      print('Epoch [{}/{}] , Step [{}/{}] , Loss: {:.4f}'.format(epoch+1 , num_epochs , i+1 , total_step , loss.item()))



# Test the model
with torch.no_grad():
  correct = 0
  total = 0
  for images , labels in test_loader:
    images = images.reshape(-1 , sequence_length , input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)
    _ , predicted = torch.max(outputs.data , 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test Accuracy of the model on the 10000 test images : {}%'.format(100 * correct / total))


# Save the model checkpoint
torch.save(model.state_dict() , 'model.ckpt')

Epoch [1/5] , Step [100/600] , Loss: 0.5092
Epoch [1/5] , Step [200/600] , Loss: 0.3038
Epoch [1/5] , Step [300/600] , Loss: 0.4030
Epoch [1/5] , Step [400/600] , Loss: 0.2759
Epoch [1/5] , Step [500/600] , Loss: 0.1604
Epoch [1/5] , Step [600/600] , Loss: 0.1320
Epoch [2/5] , Step [100/600] , Loss: 0.0562
Epoch [2/5] , Step [200/600] , Loss: 0.0654
Epoch [2/5] , Step [300/600] , Loss: 0.0637
Epoch [2/5] , Step [400/600] , Loss: 0.0206
Epoch [2/5] , Step [500/600] , Loss: 0.1433
Epoch [2/5] , Step [600/600] , Loss: 0.1403
Epoch [3/5] , Step [100/600] , Loss: 0.0051
Epoch [3/5] , Step [200/600] , Loss: 0.0657
Epoch [3/5] , Step [300/600] , Loss: 0.0351
Epoch [3/5] , Step [400/600] , Loss: 0.0093
Epoch [3/5] , Step [500/600] , Loss: 0.0997
Epoch [3/5] , Step [600/600] , Loss: 0.1013
Epoch [4/5] , Step [100/600] , Loss: 0.0132
Epoch [4/5] , Step [200/600] , Loss: 0.0215
Epoch [4/5] , Step [300/600] , Loss: 0.0261
Epoch [4/5] , Step [400/600] , Loss: 0.0095
Epoch [4/5] , Step [500/600] , L